# Taller 1: Extracción de Datos y Análisis de Sentimiento en E-commerce

___

**Integrantes del grupo**

 - OSCAR EDUARDO AVILA BERNAL
 - JOSE DARIO NAVAS TORRES
 - JUAN DAVID ORTEGA MURCIA
 - NICOLAS ESTEBAN ROMERO VARGAS

En este proyecto, se desarrolló una solución automatizada de extracción de datos y análisis de sentimiento aplicada al contexto de e-commerce. Utilizando técnicas de web scraping, se recolectaron datos clave de productos, como nombres, precios, calificaciones y comentarios de usuarios, con el fin de generar insights útiles para los consumidores y las empresas.

Además, se implementó un análisis de sentimiento para interpretar la percepción de los usuarios sobre los productos y una fórmula ponderada que calcula un puntaje de reputación integral. Este puntaje combina criterios como popularidad, calidad percibida y fiabilidad del vendedor, permitiendo así a los usuarios tomar decisiones más informadas.

### Tabla de Contenido

1. [Extracción de Datos con Web Scraping](#extraccion-de-datos-con-web-scraping)
2. [Análisis de Sentimientos](#Análisis-de-Sentimientos)
3. [Cálculo del Puntaje de Reputación](#modelo-de-puntaje)
4. [Conclusión](#conclusión)


## Librerias
librerías necesarias para:
- Realizar solicitudes HTTP con `requests`
- Extraer y analizar datos HTML con `BeautifulSoup`
- Manipular datos tabulares con `pandas`
- Realizar análisis de sentimientos con `TextBlob`
- Gestionar procesamiento de lenguaje natural con `nltk`
- Utilizar el traductor de Google (`googletrans`) para posibles traducciones.

In [1]:
import nltk
import requests
import pandas as pd
from bs4 import BeautifulSoup
from textblob import TextBlob
from googletrans import Translator
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

## Configuración Inicial
Se define la URL base de Mercado Libre y el número de productos a buscar.
Esto permite ajustar dinámicamente el rango de scraping.

### URL https://www.mercadolibre.com.co/ofertas

## Solicitud HTTP y Verificación
Se realiza una solicitud HTTP para acceder a la página definida.
Se utiliza `BeautifulSoup` para procesar el contenido HTML si la solicitud es exitosa.

In [59]:
Productos_buscados = 100
url = 'https://www.mercadolibre.com.co/ofertas'

In [ ]:
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")

    enlaces = soup.find_all("a")
    
else:
    print(f"Error al acceder a la página. Código de estado: {response.status_code}")
    

## Extracción de Datos con Web Scraping

Se realiza el proceso de extracción de datos desde la página de comercio electrónico seleccionada. El objetivo es recolectar información clave de los productos para construir un dataset estructurado.

Variables Iniciales

- links: Almacena los enlaces a las páginas individuales de productos.
- nombre: Lista de nombres de los productos.
- precio: Precios de los productos en formato numérico.
- vendedor: Información sobre el vendedor.
- caracteristicas: Descripción principal del producto.
- calificacion_v: Calificación del vendedor.
- comentarios: Comentarios de los usuarios sobre los productos.
- estado: Estado del producto (nuevo, usado).
- c_ventas: Cantidad de ventas realizadas.
- calificacion_p: Calificación promedio del producto.
- c_opiniones: Número de opiniones recibidas.

In [61]:
links = []
nombre = []
precio = []
vendedor = []
caracteristicas = []
calificacion_v = []
comentarios = []
estado = []
c_ventas = []
calificacion_p = []
c_opiniones = []


Productos_buscados = 100
request = requests.get(url)
soup = BeautifulSoup(request.text, "html.parser")

### Extracción de Enlaces a Productos
Se extraen los enlaces a las páginas de cada producto y se almacenan en la lista links.

In [62]:
for product in soup.find_all('div', {'class': 'items-with-smart-groups'}):
  for link in product.find_all('a', {'class': 'poly-component__title'}):
    while len(links)<= Productos_buscados:
      links.append(link.get('href'))
      
      break

### Extracción de Datos de Cada Producto
Para cada enlace en links, se realiza un nuevo scraping para extraer los detalles de los productos.

Se realiza un bucle para iterar sobre cada enlace almacenado en links. Para cada producto, se extraen diversos detalles clave como el nombre, precio, vendedor, características, calificaciones y más.
**Detalles Extraídos:**
 - Nombre del Producto: Se encuentra en el encabezado principal.
 - Precio: Se obtiene del contenedor que muestra el precio del producto.
 - Vendedor: Información sobre el vendedor y su perfil.
 - Descripción: Texto breve sobre las características principales del producto.
 - Calificación del Vendedor: Indicador de confianza basado en su desempeño.
 - Comentarios de Usuarios: Opiniones textuales sobre el producto.
 - Ventas y Estado: Número de ventas realizadas y estado (nuevo/usado).
 - Calificación del Producto: Puntuación promedio otorgada por los usuarios.
 - Cantidad de Opiniones: Número total de reseñas disponibles.

In [63]:
for entry in links:
  request = requests.get(entry)
  soup = BeautifulSoup(request.text, "html.parser")
  for product in soup.find_all('h1', {'class':'ui-pdp-title'}):
    if product:
        nombre.append(product.text.strip())
    else:
        print(f"[AVISO] No se encontró el nombre del producto en: {entry}")
    
    
  for valor in soup.find_all('div', {'class':'ui-pdp-price__second-line'}):
    for price in valor.find_all('span', {'class':'andes-money-amount__fraction'}):
      if price:
            precios = int(price.text.strip().replace('.', ''))
            precio.append(precios)
      else:
          print(f"[AVISO] No se encontró el precio en: {entry}")
      
      
  for seller in soup.find_all('span', {'class':'ui-pdp-seller__label-text-with-icon'}):
    vendedor.append(seller.text)
  for seller_2 in soup.find_all('div', {'class':'ui-pdp-seller__header__title'}):
    for seller_3 in seller_2.find_all('span', {'class':""}):
      if seller_3:
                vendedor.append(seller_3.text.strip())
      else:
                print(f"[AVISO] No se encontró el vendedor en: {entry}")

  
  for description in soup.find_all('p', {'class':'ui-pdp-description__content'}):
    if description:
        caracteristicas.append(description.text.strip())
    else:
        print(f"[AVISO] No se encontró la descripción en: {entry}")

  
  for rating in soup.find_all('ul', {'class':'ui-seller-data-status__thermometer thermometer-large'}):
    if rating:
        calificacion_v.append(rating.get('value'))
    else:
        print(f"[AVISO] No se encontró la calificación en: {entry}")
    
    
  for comments in soup.find_all('div', {'class':'ui-review-capability-comments'}):
    particion = []
    if comments:
      for comments_2 in comments.find_all('div'):
        for comments_3 in comments_2.find_all('p', {'class':'ui-review-capability-comments__comment__content ui-review-capability-comments__comment__content'}):
          particion.append(comments_3.text.strip())
      comentarios.append(' '.join(particion))
    else:
        print(f"[AVISO] No se encontró la sección de comentarios en: {entry}")
  
  
  for sells in soup.find_all('span', {'class': 'ui-pdp-subtitle'}):
    texto = sells.text.strip()
    if "|" in texto:
        partes = texto.split('|')
        estado_v = partes[0].strip()
        ventas = partes[1].strip()
        ventas = ventas.replace('+', '').replace('vendidos', '').strip()
        if "mil" in ventas:
            ventas = ventas.replace("mil", "").strip()
            if ventas == "":
                ventas = 1000
            else:
                ventas = int(ventas) * 1000
        else:
            try:
                ventas = int(ventas)
            except ValueError:
                print(f"[AVISO] No se pudo convertir ventas a número: {ventas}")
                ventas = 0
        estado.append(estado_v)
        c_ventas.append(ventas)
    else:
        print(f"[AVISO] No se encontró el delimitador '|' en el texto: {texto}")
    
    
  for valoration in soup.find_all('span', {'class':'ui-pdp-review__rating'}):
    if valoration:
      calificacion_p.append(valoration.text)
    else:
        print(f"[AVISO] No se encontró la sección de comentarios en: {entry}")
  
    
  for opinions in soup.find_all('span', {'class':'ui-pdp-review__amount'}):
    if opinions:
      c_opiniones.append(opinions.text.strip('()'))
    else:
        print(f"[AVISO] No se encontró la sección de comentarios en: {entry}")

### Creación del Dataset
Se consolidan todos los datos en un DataFrame de pandas y se exportan a un archivo CSV llamado ecommerce.csv para análisis posterior.

In [64]:
product_df = pd.DataFrame(zip(nombre,precio,c_ventas,calificacion_p,c_opiniones,vendedor,caracteristicas,calificacion_v,comentarios),
                          columns = ['Nombre', 'Precio','Cantidad Ventas','Calificacion Producto','Cantidad Opiniones', 'Vendedor', 'Descripcion', 'Rating Vendedor', 'Comentarios'])
product_df.to_csv("ecommerce.csv",index=False)

In [65]:
product_df.head()

,Nombre,Precio,Cantidad Ventas,Calificacion Producto,Cantidad Opiniones,Vendedor,Descripcion,Rating Vendedor,Comentarios
0,Samsung Galaxy S24 Ultra 5g Dual Sim 512 Gb Ti...,4489900,50,5.0,14,ITECHNOLOGY,Fotografía profesional en tu bolsilloDescubre ...,5,"Excelente equipo!!. Funciona genial, apenas lo..."
1,Nintendo Switch Oled 64gb Standard Color Rojo ...,1274000,50,4.8,13,ROALEGAMES,La consola Switch ofrece entretenimiento diari...,5,Es la version japonesa pero eso no influye en ...
2,Gopro Hero13 Black + Accesory Bundle,1778400,5,4.9,575,COMERCIALIZADORA JYT,El vendedor no incluyó una descripción del pro...,5,Nunca tuve la oportunidad de usar unos audífon...
3,Audífonos Sony Bluetooth Noise Cancelling | Wh...,1184900,1000,4.6,372,SONY COLOMBIA,"Sony, sin lugar a dudas es una de las marcas m...",5,"La tercera que compro, por algo será!. Las me..."
4,Almohada Emma Memory Adapt | Memory Foam | Esp...,181293,1000,4.9,66,Emmasleepsasemmasleepsas,Cada dispositivo Refurbi ha pasado por un rigu...,5,Me encanta 💖.


In [68]:
df_total = pd.read_csv('ecommerce.csv')

Se importa el archivo generado (ecommerce.csv) para su análisis. Una vez cargado, se verifica la presencia de valores nulos en el dataset para identificar posibles inconsistencias o datos faltantes que puedan afectar el análisis posterior.

In [69]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Nombre                 38 non-null     object 
 1   Precio                 38 non-null     int64  
 2   Cantidad Ventas        38 non-null     int64  
 3   Calificacion Producto  38 non-null     float64
 4   Cantidad Opiniones     38 non-null     int64  
 5   Vendedor               38 non-null     object 
 6   Descripcion            38 non-null     object 
 7   Rating Vendedor        38 non-null     int64  
 8   Comentarios            37 non-null     object 
dtypes: float64(1), int64(4), object(4)
memory usage: 2.8+ KB


In [71]:
no_null = df_total[df_total["Comentarios"].notna()]
data = no_null.reset_index()
df = data.drop(columns=['index'])

In [72]:
df.head()

,Nombre,Precio,Cantidad Ventas,Calificacion Producto,Cantidad Opiniones,Vendedor,Descripcion,Rating Vendedor,Comentarios
0,Samsung Galaxy S24 Ultra 5g Dual Sim 512 Gb Ti...,4489900,50,5.0,14,ITECHNOLOGY,Fotografía profesional en tu bolsilloDescubre ...,5,"Excelente equipo!!. Funciona genial, apenas lo..."
1,Nintendo Switch Oled 64gb Standard Color Rojo ...,1274000,50,4.8,13,ROALEGAMES,La consola Switch ofrece entretenimiento diari...,5,Es la version japonesa pero eso no influye en ...
2,Gopro Hero13 Black + Accesory Bundle,1778400,5,4.9,575,COMERCIALIZADORA JYT,El vendedor no incluyó una descripción del pro...,5,Nunca tuve la oportunidad de usar unos audífon...
3,Audífonos Sony Bluetooth Noise Cancelling | Wh...,1184900,1000,4.6,372,SONY COLOMBIA,"Sony, sin lugar a dudas es una de las marcas m...",5,"La tercera que compro, por algo será!. Las me..."
4,Almohada Emma Memory Adapt | Memory Foam | Esp...,181293,1000,4.9,66,Emmasleepsasemmasleepsas,Cada dispositivo Refurbi ha pasado por un rigu...,5,Me encanta 💖.


## Análisis de Sentimientos

En esta sección, se analiza el sentimiento de los comentarios extraídos utilizando técnicas de procesamiento de lenguaje natural (NLP). Se realizan los siguientes pasos:

1. **Inicialización de Variables**:
   - Listas para almacenar valores iniciales y finales de polaridad y subjetividad:
     - `polaridad_inicial`, `polaridad_final`
     - `subjetividad_inicial`, `subjetividad_final`

2. **Traducción y Análisis Inicial**:
   - Los comentarios en español son traducidos al inglés usando la biblioteca `Translator`.
   - Se calcula la polaridad y subjetividad iniciales con `TextBlob`.

3. **Procesamiento de Texto**:
   - **Tokenización**: Se separa cada comentario en palabras individuales.
   - **Stemmer y Lemmatizer**: Se aplica la normalización de palabras para reducirlas a sus raíces y formas base.
   - **Eliminación de Stopwords**: Se eliminan palabras irrelevantes como artículos y preposiciones.

4. **Reconstrucción y Análisis Final**:
   - Se reconstruyen las frases procesadas y se realiza un nuevo análisis de sentimientos.
   - Se obtienen valores finales de polaridad y subjetividad.

5. **Creación del Dataset**:
   - Los resultados se almacenan en un DataFrame llamado `sentimiento_df` con las columnas:
     - `Polaridad Inicial`, `Subjetividad Inicial`
     - `Polaridad Final`, `Subjetividad Final`


In [ ]:
polaridad_inicial = []
polaridad_final = []
subjetividad_inicial = []
subjetividad_final = []


stemmer = PorterStemmer()
stemmer2 = SnowballStemmer(language='spanish')
lemmatizer = WordNetLemmatizer()
translator = Translator()
frases = df['Comentarios']
for frase in frases:
  
  blob = TextBlob(frase)
  translation = translator.translate(str(blob), src='es', dest='en').text
  
  
  blob_t = TextBlob(translation)
  sentimiento = blob_t.sentiment
  polaridad = sentimiento.polarity
  subjetividad = sentimiento.subjectivity
  polaridad_inicial.append(polaridad)
  subjetividad_inicial.append(subjetividad)

  tokens=word_tokenize(translation.lower())
  prueba_2 = [stemmer2.stem(palabra_2) for palabra_2 in tokens]
  
  prueba_3 = [lemmatizer.lemmatize(palabra_3) for palabra_3 in tokens]
  
  stopwords = nltk.corpus.stopwords.words('english')
  newtokens=[word for word in tokens if word not in stopwords]

  sentence = ' '.join(newtokens)
  
  translation_2 = translator.translate(str(sentence), src='en', dest='es').text

  blob_t = TextBlob(sentence)
  sentimiento_t = blob_t.sentiment
  polaridad_t = sentimiento_t.polarity
  subjetividad_t = sentimiento_t.subjectivity
  polaridad_final.append(polaridad_t)
  subjetividad_final.append(subjetividad_t)
sentimiento_df = pd.DataFrame(zip(polaridad_inicial,subjetividad_inicial,polaridad_final,subjetividad_final), 
                              columns=['Polaridad Inicial','Subjetividad Inicial','Polaridad Final','Subjetividad Final'])

In [74]:
df_final = pd.concat([df,sentimiento_df],axis=1)
df_final.head()

,Nombre,Precio,Cantidad Ventas,Calificacion Producto,Cantidad Opiniones,Vendedor,Descripcion,Rating Vendedor,Comentarios,Polaridad Inicial,Subjetividad Inicial,Polaridad Final,Subjetividad Final
0,Samsung Galaxy S24 Ultra 5g Dual Sim 512 Gb Ti...,4489900,50,5.0,14,ITECHNOLOGY,Fotografía profesional en tu bolsilloDescubre ...,5,"Excelente equipo!!. Funciona genial, apenas lo...",0.700000,0.937500,0.866667,0.916667
1,Nintendo Switch Oled 64gb Standard Color Rojo ...,1274000,50,4.8,13,ROALEGAMES,La consola Switch ofrece entretenimiento diari...,5,Es la version japonesa pero eso no influye en ...,0.075000,0.325000,0.075000,0.325000
2,Gopro Hero13 Black + Accesory Bundle,1778400,5,4.9,575,COMERCIALIZADORA JYT,El vendedor no incluyó una descripción del pro...,5,Nunca tuve la oportunidad de usar unos audífon...,0.345714,0.436217,0.368421,0.439766
3,Audífonos Sony Bluetooth Noise Cancelling | Wh...,1184900,1000,4.6,372,SONY COLOMBIA,"Sony, sin lugar a dudas es una de las marcas m...",5,"La tercera que compro, por algo será!. Las me...",0.446667,0.453333,0.433333,0.441667
4,Almohada Emma Memory Adapt | Memory Foam | Esp...,181293,1000,4.9,66,Emmasleepsasemmasleepsas,Cada dispositivo Refurbi ha pasado por un rigu...,5,Me encanta 💖.,0.500000,0.600000,0.500000,0.600000


## Modelo de Puntaje
Esta fórmula ponderada calcula un puntaje de reputación para cada producto. Este puntaje tiene como objetivo ayudar a los usuarios a evaluar de manera más completa y objetiva la calidad y confiabilidad de los productos en función de múltiples criterios.

$$
Puntaje = 10 \times \left( w1 \times \left( \frac{Cantidad Ventas}{max\_ventas} \right) + w2 \times \left( \frac{Calificacion Producto}{5} \right) + w3 \times \left( \frac{Cantidad Opiniones}{max\_opiniones} \right) + w4 \times \left( \frac{Rating Vendedor}{5} \right) + w5 \times \left( \frac{Polaridad Final + 1}{2} \right) - w6 \times Subjetividad Final \right)
$$



**Desglose de los Componentes**

1. **Cantidad Ventas**:
   - Se normaliza dividiendo el número de ventas de un producto entre la cantidad máxima de ventas en la muestra.
   - Contribuye a reflejar la popularidad del producto.

2. **Calificación Producto**:
   - Es la puntuación del producto (generalmente en una escala de 1 a 5) y se normaliza dividiéndola entre 5.
   - Representa la percepción de calidad por parte de los compradores.

3. **Cantidad Opiniones**:
   - Se normaliza dividiendo entre la cantidad máxima de opiniones en la muestra.
   - Indica cuánta información está disponible sobre el producto, lo cual es útil para determinar su confiabilidad.

4. **Rating Vendedor**:
   - Se normaliza en una escala de 1 a 5.
   - Refleja la fiabilidad del vendedor con base en su historial.

5. **Polaridad Final**:
   - Calculada a partir del análisis de sentimiento, se ajusta a una escala entre 0 y 1 al sumar 1 y dividir por 2.
   - Refleja si los comentarios de los usuarios son mayormente positivos o negativos.

6. **Subjetividad Final**:
   - Penaliza los productos con comentarios subjetivos, ya que estos suelen ser menos informativos o sesgados.

7. **Pesos (w1, w2, ..., w6)**:
   - Asignados a cada componente para ajustar su impacto en el puntaje final según su importancia relativa.

**Nota**: El puntaje final está escalado en una escala de 0 a 10 para facilitar su interpretación.

In [77]:
w1, w2, w3, w4, w5, w6 = 0.22, 0.2, 0.15, 0.2, 0.23, 0.1

max_ventas = df_final['Cantidad Ventas'].max()
max_opiniones = df_final['Cantidad Opiniones'].max()

df_final['Puntaje'] = 10 * (
    w1 * (df_final['Cantidad Ventas'] / max_ventas) +
    w2 * (df_final['Calificacion Producto'] / 5) +
    w3 * (df_final['Cantidad Opiniones'] / max_opiniones) +
    w4 * (df_final['Rating Vendedor'] / 5) +
    w5 * ((df_final['Polaridad Final'] + 1) / 2) -
    w6 * df_final['Subjetividad Final']
)

top_10_productos = df_final[['Nombre', 'Puntaje', 'Precio']].sort_values(by='Puntaje', ascending=False).head(10)

pivot_table = top_10_productos.pivot_table(index='Nombre', values=['Puntaje', 'Precio'], aggfunc='mean').sort_values(by='Puntaje', ascending=False)

In [78]:
pivot_table

,Precio,Puntaje
Nombre,,
Silla Ejecutiva Magnux Ergonómica Negra Con Tapizado Mesh Color Negro,154900.0,7.281992
"Cortina Blackout 100% Anti Luz, 2 Paños 140x220 Textura Lino",125990.0,6.366771
Mesa Portafolio Plegable 180cm Resistente Exterior Jardin Color Blanco,164600.0,6.220411
Cortinas Blackout 100% Casatua 2paños 140x220 Lino Doble Faz,99990.0,6.101062
Lavaplatos Moderno Acero Inoxidable De Lujo Inteligente,1386000.0,5.607421
Tenis Cloudfoam Comfy Con Cordones Elásticos Ih2966 adidas,174965.0,5.448011
Audífonos Sony Bluetooth Noise Cancelling | Wh-1000xm5 Color Negro,1184900.0,5.425009
Audífonos Inalámbricos Con Noise Cancelling Wh-ch720n Color Negro,327959.0,5.412123
Xiaomi Redmi Watch 5 Active Con Hyperos Bateria 18 Días Llamadas Ipx8 Negro,147348.0,5.403816


### Conclusión

El análisis realizado permitió calcular un puntaje de reputación para cada producto basado en múltiples criterios ponderados, como ventas, calificaciones, opiniones y análisis de sentimientos. La tabla final presenta una clasificación que combina el precio del producto con su respectivo puntaje, proporcionando a los usuarios una herramienta clara y efectiva para evaluar opciones.

#### Observaciones:
1. **Relevancia del Puntaje**:
   - Productos con puntajes altos combinan buenos niveles de ventas, calificaciones positivas y una percepción favorable en los comentarios.

2. **Relación Precio-Puntaje**:
   - Se observa que productos con precios moderados pueden tener puntajes altos debido a su aceptación general y percepción positiva por parte de los usuarios.

3. **Aplicabilidad del Modelo**:
   - Este sistema de puntaje puede ser una herramienta útil para priorizar productos en recomendaciones o análisis de inventarios.

En resumen, este análisis representa un enfoque integrador que combina datos cuantitativos y cualitativos para generar valor en la toma de decisiones de los usuarios y empresas en el sector de e-commerce.